# Notes


In [1]:
import pandas as pd
import fields

In [2]:
df = pd.read_parquet("./us_births_20251028A.parquet", engine="pyarrow").convert_dtypes()

c:\Users\FrankBuckley\miniconda3\envs\data-stats-models-ml\Lib\site-packages\pandas\core\dtypes\cast.py:1060: RuntimeWarning: invalid value encountered in cast
  if (arr.astype(int) == arr).all():
c:\Users\FrankBuckley\miniconda3\envs\data-stats-models-ml\Lib\site-packages\pandas\core\dtypes\cast.py:1084: RuntimeWarning: invalid value encountered in cast
  if (arr.astype(int) == arr).all():


In [3]:
import columns
import fields


df[fields.DOWNS] = pd.NA
df[fields.UCA_DOWNS] = pd.NA

columns.add_computed_columns(df)
columns.set_imported_column_types(df)
df.describe()

,dob_yy,dob_mm,bfacil,f_bfacil,mager,mager14,mager9,mbstate_rec,mrace31,mrace6,...,f_ca_chrom,f_ca_hypos,no_congen,f_mpcb,precare5,previs,previs_rec,f_tpcv,f_wic,ds_lb_chance
count,57622691.0,57622691.0,54342108.0,10919546.0,57622691.0,57622691.0,57622691.0,41576190.0,41444648.0,41444648.0,...,57622691.0,57622691.0,41364225.0,57622691.0,41364225.0,41364225.0,57622691.0,57622691.0,57622691.0,5.762269e+07
mean,2016.680317,6.575162,1.037569,0.997558,28.725319,9.28653,4.345362,1.232839,1.637463,6.676241,...,0.935972,0.935972,-1745120756965840106588833547970714374406075492...,0.935969,1.406207,13.247035,6.88725,0.997755,0.93597,2.070002e-03
std,4.934429,3.628645,0.325031,0.04936,5.92435,1.380796,1.219313,0.427762,1.990175,9.539811,...,0.244803,0.244803,inf,0.244997,0.862332,14.087228,2.076918,0.0479,0.244923,3.287053e-03
min,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,-1336769771172320803233997070482570618156440084...,-32.15625,0.0,0.0,0.0,-31.9375,-32.6875,6.552215e-04
25%,2013.0,4.0,1.0,1.0,24.0,8.0,3.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,9.0,6.0,1.0,1.0,7.234341e-04
50%,2017.0,7.0,1.0,1.0,29.0,9.0,4.0,1.0,1.0,2.0,...,1.0,1.0,1.0,1.0,1.0,11.0,7.0,1.0,1.0,9.557278e-04
75%,2021.0,9.0,1.0,1.0,33.0,10.0,5.0,1.0,2.0,10.0,...,1.0,1.0,1.0,1.0,1.0,13.0,8.0,1.0,1.0,1.746650e-03
max,2024.0,2240.0,9.0,1.0,50.0,14.0,9.0,3.0,99.0,99.0,...,1.0,1.0,9.0,1.0,5.0,99.0,12.0,1.0,1.0,3.799470e-02


In [4]:
df_grouped = df.groupby(fields.CA_DOWN).size().reset_index(name="counts")
df_grouped = df_grouped.set_index(fields.CA_DOWN)
df_grouped

,counts
ca_down,
C,6144
N,25705006
P,7453
U,49759
Y,54


In [5]:
df_grouped_by_year = (
    df.groupby([fields.DOB_YY, fields.CA_DOWN]).size().reset_index(name="counts")
)
df_pivot = df_grouped_by_year.pivot(
    index=fields.DOB_YY, columns=fields.CA_DOWN, values="counts"
)
df_pivot

ca_down,C,N,P,U,Y
dob_yy,,,,,
0.0,NaN,NaN,NaN,NaN,54.0
2018.0,921.0,3793292.0,1187.0,6107.0,NaN
2019.0,923.0,3750057.0,1107.0,5468.0,NaN
2020.0,894.0,3612707.0,1060.0,5165.0,NaN
2021.0,857.0,3660477.0,1046.0,7548.0,NaN
2022.0,818.0,3666667.0,1043.0,7501.0,NaN
2023.0,867.0,3594893.0,988.0,8333.0,NaN
2024.0,864.0,3626913.0,1022.0,9637.0,NaN


In [6]:
# group by DOB_YY sum DS_LB_CHANCE
df_grouped_sum = (
    df.groupby(fields.DOB_YY)
    .agg(total_ds_lb_chance=pd.NamedAgg(column=fields.DS_LB_CHANCE, aggfunc="sum"))
    .reset_index()
)
df_grouped_sum

,dob_yy,total_ds_lb_chance
0,0.0,0.035382
1,2009.0,7565.147534
2,2010.0,7439.844521
3,2012.0,7500.288505
4,2013.0,7568.050772
5,2014.0,7773.290827
6,2015.0,7888.163531
7,2016.0,7977.655248
8,2017.0,7968.123687
9,2018.0,8023.239877
